In [1]:
# | output: false
# | echo: false
import builtins
import rich

::: {.callout-note}

This page leverages some basic transform syntax from later parts of the walkthrough. Don't worry too much about it for now: the core purpose of this page is to understand how relations work in Vinyl.

:::

## Properties

As in SQL, relations are the core data model. But relations in Vinyl have some unique properties:


### Relations are column aware

Take the `seattle_weather` dataset from vinyl's examples. 

In [2]:
from vinyl.examples import seattle_weather

weather = seattle_weather()

`weather` is a Vinytable object and carries far more information that a standard sql table or cte. The schema, for example, can be pulled easily without running a query against the database. 


<Card>

In [3]:
print(weather.schema())

┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ # ┃ column        ┃ type         ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ 0 │ date          │ timestamp(6) │
│ 1 │ precipitation │ float64      │
│ 2 │ temp_max      │ float64      │
│ 3 │ temp_min      │ float64      │
│ 4 │ wind          │ float64      │
│ 5 │ weather       │ string       │
└───┴───────────────┴──────────────┘

</Card>

Columns are attrributes of the class, so the code below returns the `wind` column.


<Card>

In [4]:
weather.wind

r0 := DatabaseTable: seattle_weather
  date          timestamp(6)
  precipitation float64
  temp_max      float64
  temp_min      float64
  wind          float64
  weather       string

wind: r0.wind

</Card>

And, after a basic transform:


<Card>

In [5]:
wind_doubled = weather.select(weather.wind * 2)
wind_doubled

r0 := DatabaseTable: seattle_weather
  date          timestamp(6)
  precipitation float64
  temp_max      float64
  temp_min      float64
  wind          float64
  weather       string

Project[r0]
  Multiply(wind, 2): r0.wind * 2

</Card>

This means you can access the information at any point in your queries. For example


<Card>

In [6]:
temps = weather.select([col for col in weather.columns if col.startswith("temp")])
print(temps.schema())

┏━━━┳━━━━━━━━━━┳━━━━━━━━━┓
┃ # ┃ column   ┃ type    ┃
┡━━━╇━━━━━━━━━━╇━━━━━━━━━┩
│ 0 │ temp_max │ float64 │
│ 1 │ temp_min │ float64 │
└───┴──────────┴─────────┘

</Card>

will select the temperature columns from `weather`.

### Relations are lazy

Vinyl keeps track of its syntax lazily, and only compiles when a variable is executed or its sql is generated.

By default, vinyl will return a string representation of the ast associated with a table unless specified otherwise. For example:


<Card>

In [7]:
temps

r0 := DatabaseTable: seattle_weather
  date          timestamp(6)
  precipitation float64
  temp_max      float64
  temp_min      float64
  wind          float64
  weather       string

Project[r0]
  temp_max: r0.temp_max
  temp_min: r0.temp_min

</Card>

If you'd like to see a graphical representation, use `.visualize()`:


<Card>

In [8]:
temps.visualize()

╭──────────────────────╮     
                                 │ Project              │     
         ╭──────────────────────▶│    temp_max: float64 │     
         │                       │    temp_min: float64 │     
         │                       ╰──────────────────────╯     
         │                                  ▲▲                
         │                                  ││                
         │                                  ││                
         │                                  ││                
╭─────────────────╮                ╭─────────────────╮        
│ temp_max: Field │                │ temp_min: Field │        
│    :: float64   │                │    :: float64   │        
╰─────────────────╯                ╰─────────────────╯        
         ▲                                  ▲│                
         │                                  ││                
         │                                  ││                
         │                                  ││                
         │                                  ││                
         │                  ╭────────────────────────────────╮
         │                  │ seattle_weather: DatabaseTable │
         │                  │    date: timestamp(6)          │
         │                  │    precipitation: float64      │
         ╰──────────────────│    temp_max: float64           │
                            │    temp_min: float64           │
                            │    wind: float64               │
                            │    weather: string             │
                            ╰────────────────────────────────╯

</Card>

To execute this, you only need to run:


<Card>

In [9]:
temps.execute()

,temp_max,temp_min
0,12.8,5.0
1,10.6,2.8
2,11.7,7.2
3,12.2,5.6
4,8.9,2.8
...,...,...
1456,4.4,1.7
1457,5.0,1.7
1458,7.2,0.6
1459,5.6,-1.0


</Card>

By default, this returns a pandas DataFrame. You can also return a text or pyarrow representation by specifying that as in the examples below:


<Card>

In [10]:
temps.execute("text")

┏━━━━━━━━━━┳━━━━━━━━━━┓
┃ temp_max ┃ temp_min ┃
┡━━━━━━━━━━╇━━━━━━━━━━┩
│ float64  │ float64  │
├──────────┼──────────┤
│     12.8 │      5.0 │
│     10.6 │      2.8 │
│     11.7 │      7.2 │
│     12.2 │      5.6 │
│      8.9 │      2.8 │
│      4.4 │      2.2 │
│      7.2 │      2.8 │
│     10.0 │      2.8 │
│      9.4 │      5.0 │
│      6.1 │      0.6 │
│        … │        … │
└──────────┴──────────┘

</Card>

<Card>

In [11]:
temps.execute("pyarrow")

pyarrow.Table
temp_max: double
temp_min: double
----
temp_max: [[12.8,10.6,11.7,12.2,8.9,...,4.4,5,7.2,5.6,5.6]]
temp_min: [[5,2.8,7.2,5.6,2.8,...,1.7,1.7,0.6,-1,-2.1]]

</Card>

You can also execute it and save the results to various formats (csv, json, etc.) using `.save()`.

### Relations are selectively mutable

By default, VinylTables are **immutable**. For example, the original table ast is printed when run is the same as the original `weather`.


<Card>

In [12]:
weather.select(weather.wind * 2)
weather

DatabaseTable: seattle_weather
  date          timestamp(6)
  precipitation float64
  temp_max      float64
  temp_min      float64
  wind          float64
  weather       string

</Card>

That said, there are two key cases where VinylTables are considered mutable. This allows for a more ergonomic syntax, especially when you are chaining several transforms together.

The two cases are:
1. Within specially decorated functions (i.e. those with `@model` or `@metric` decorator)
2. Context managers

The first case is designed to support <Tooltip tip="more on this in the quickstart">pipelines</Tooltip>:


<Card>

In [13]:
from vinyl import T, model


@model(deps=[seattle_weather])
def wind(w: T) -> T:
    w.select(w.wind * 2)
    return w


wind()

r0 := DatabaseTable: seattle_weather
  date          timestamp(6)
  precipitation float64
  temp_max      float64
  temp_min      float64
  wind          float64
  weather       string

Project[r0]
  Multiply(wind, 2): r0.wind * 2

</Card>

The second use case is designed primarily for analysis use cases (i.e. in a notebook):


<Card>

In [14]:
with seattle_weather() as w:
    w.select(w.wind * 2)

w

r0 := DatabaseTable: seattle_weather
  date          timestamp(6)
  precipitation float64
  temp_max      float64
  temp_min      float64
  wind          float64
  weather       string

Project[r0]
  Multiply(wind, 2): r0.wind * 2

</Card>

This approach can also be used inside vinyl pipeline functions to create a sort of ["cte"](https://learnsql.com/blog/what-is-common-table-expression/).

True CTEs don't exist in vinyl. In immutable mode, you simply need to save a subcalculation to a variable name to achieve the same functionality. But in mutable mode, as in a pipeline function, saving subcalculations as variables can cause issues, because it may have off-target effects on the sources of the subcalculation.

This is where the context manager comes in handy. It allows you to create a temporary variable that is only available within the context of the context manager.

In the example below, we get the average wind by month and then use it to calculate the wind speed deviation from the average.

::: {.callout-note}
This can be done more efficiently in vinyl using a window function, but this is a good example of how to use the context manager.
:::

<Card>

In [15]:
from vinyl import T, model, join


@model(deps=[seattle_weather])
def wind_vs_avg(w: T) -> T:
    with w as mw:
        mw.aggregate(
            cols = {"avg_wind": mw.wind.mean()}, 
            by = {"month": mw.date.dt.floor(months=1)}
        )

    
    j = join(w, mw, on = [w.date.dt.floor(months=1) == mw.month])
    j.select([w.date, w.wind - mw.avg_wind])

    return j

wind_vs_avg().execute("text")

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ date                ┃ Subtract(wind, avg_wind) ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ timestamp(6)        │ float64                  │
├─────────────────────┼──────────────────────────┤
│ 2012-01-01 00:00:00 │                      0.8 │
│ 2012-01-02 00:00:00 │                      0.6 │
│ 2012-01-03 00:00:00 │                     -1.6 │
│ 2012-01-04 00:00:00 │                      0.8 │
│ 2012-01-05 00:00:00 │                      2.2 │
│ 2012-01-06 00:00:00 │                     -1.7 │
│ 2012-01-07 00:00:00 │                     -1.6 │
│ 2012-01-08 00:00:00 │                     -1.9 │
│ 2012-01-09 00:00:00 │                     -0.5 │
│ 2012-01-10 00:00:00 │                     -0.5 │
│ …                   │                        … │
└─────────────────────┴──────────────────────────┘

</Card>

### Relations are dialect-independent

Vinyl uses the Ibis library to generate SQL. This means that you can write your queries in a dialect agnostic way. For example, the following code works in across the dialects currently supported by Vinyl:

- BigQuery
- Snowflake
- DuckDB
- Postgres

Ibis itself supports almost 20 dialects, so we plan to add more over time.

As an example, let's create a table that lists the max temperature of each day by month:

<Card>

In [16]:
with seattle_weather() as w:
    w.aggregate(
        cols = {"temp_max": w.temp_max.collect()}, 
        sort = {"month": w.date.dt.floor(months=1)}
    )

temp_by_month = w
temp_by_month.execute("text")

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ month               ┃ temp_max       ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ timestamp           │ array<float64> │
├─────────────────────┼────────────────┤
│                     │ [              │
│                     │     12.8,      │
│                     │     10.6,      │
│                     │     11.7,      │
│                     │     12.2,      │
│                     │     8.9,       │
│                     │     4.4,       │
│                     │     7.2,       │
│                     │     10.0,      │
│                     │     9.4,       │
│                     │     6.1,       │
│                     │     6.1,       │
│                     │     6.1,       │
│                     │     5.0,       │
│                     │     4.4,       │
│                     │     1.1,       │
│ 2012-01-01 00:00:00 │     1.7,       │
│                     │     3.3,       │
│                     │     0.0,       │
│                     │     -1.1,      │
│                     │     7.2,       │
│                     │     8.3,       │
│                     │     6.7,       │
│                     │     8.3,       │
│                     │     10.0,      │
│                     │     8.9,       │
│                     │     8.9,       │
│                     │     6.7,       │
│                     │     6.7,       │
│                     │     9.4,       │
│                     │     8.3,       │
│                     │     9.4        │
│                     │ ]              │
│                     │ [              │
│                     │     8.9,       │
│                     │     8.3,       │
│                     │     14.4,      │
│                     │     15.6,      │
│                     │     13.9,      │
│                     │     16.1,      │
│                     │     15.6,      │
│                     │     10.0,      │
│                     │     11.1,      │
│                     │     12.8,      │
│                     │     8.9,       │
│                     │     8.3,       │
│                     │     7.2,       │
│                     │     6.7,       │
│ 2012-02-01 00:00:00 │     7.2,       │
│                     │     7.2,       │
│                     │     10.0,      │
│                     │     6.7,       │
│                     │     6.7,       │
│                     │     7.8,       │
│                     │     10.0,      │
│                     │     10.0,      │
│                     │     8.3,       │
│                     │     6.7,       │
│                     │     7.2,       │
│                     │     5.0,       │
│                     │     6.7,       │
│                     │     6.7,       │
│                     │     5.0        │
│                     │ ]              │
│                     │ [              │
│                     │     6.1,       │
│                     │     6.7,       │
│                     │     12.2,      │
│                     │     10.6,      │
│                     │     7.8,       │
│                     │     6.7,       │
│                     │     8.9,       │
│                     │     15.6,      │
│                     │     9.4,       │
│                     │     7.2,       │
│                     │     6.7,       │
│                     │     8.3,       │
│                     │     5.6,       │
│                     │     7.8,       │
│                     │     11.1,      │
│ 2012-03-01 00:00:00 │     8.9,       │
│                     │     10.0,      │
│                     │     5.0,       │
│                     │     7.2,       │
│                     │     7.8,       │
│                     │     8.9,       │
│                     │     10.0,      │
│                     │     12.2,      │
│                     │     15.0,      │
│                     │     13.3,      │
│                     │     12.8,      │
│                     │     14.4,      │
│                     │

</Card>

Here's how vinyl translates the query to each dialect:

<Card>

::: {.panel-tabset}

#### BigQuery

In [17]:
# | echo: false

from vinyl import original_print

original_print(temp_by_month.to_sql("bigquery"))

SELECT
  `t1`.`month`,
  `t1`.`temp_max`
FROM (
  SELECT
    TIMESTAMP_TRUNC(`t0`.`date`, MONTH) AS `month`,
    ARRAY_AGG(`t0`.`temp_max` IGNORE NULLS) AS `temp_max`
  FROM `seattle_weather` AS `t0`
  GROUP BY
    1
) AS `t1`
ORDER BY
  `t1`.`month` ASC NULLS LAST


#### Snowflake

In [18]:
# | echo: false
print(temp_by_month.to_sql("snowflake"))

SELECT
  "t1"."month",
  "t1"."temp_max"
FROM (
  SELECT
    DATE_TRUNC('MONTH', "t0"."date") AS "month",
    ARRAY_AGG(COALESCE("t0"."temp_max", PARSE_JSON('null'))) AS "temp_max"
  FROM "seattle_weather" AS "t0"
  GROUP BY
    1
) AS "t1"
ORDER BY
  "t1"."month" ASC

#### DuckDB

In [19]:
# | echo: false
print(temp_by_month.to_sql("duckdb"))

SELECT
  "t1"."month",
  "t1"."temp_max"
FROM (
  SELECT
    DATE_TRUNC('MONTH', "t0"."date") AS "month",
    ARRAY_AGG("t0"."temp_max") AS "temp_max"
  FROM "seattle_weather" AS "t0"
  GROUP BY
    1
) AS "t1"
ORDER BY
  "t1"."month" ASC

#### Postgres

In [20]:
# | echo: false
print(temp_by_month.to_sql("postgres"))

SELECT
  "t1"."month",
  "t1"."temp_max"
FROM (
  SELECT
    DATE_TRUNC('MONTH', "t0"."date") AS "month",
    ARRAY_AGG("t0"."temp_max") AS "temp_max"
  FROM "seattle_weather" AS "t0"
  GROUP BY
    1
) AS "t1"
ORDER BY
  "t1"."month" ASC

:::

</Card>

## Creating relations

In the context of a project, relations are created and managed automatically using the [source generation process](../concepts/sources).

In the context of a notebook or analysis, you can create relations from a variety of sources, including:
- Pandas DataFrames
- Polars DataFrames
- PyArrow Tables
- CSVs
- JSONs
- Parquet files

For in-memory sources, use `VinylTable.from_memory` as:

<Card>

In [21]:
from vinyl import VinylTable
import pandas as pd


d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)

VinylTable.from_memory(df)

InMemoryTable
  data:
    PandasDataFrameProxy:
         col1  col2
      0     1     3
      1     2     4

</Card>

And for file-based sources, use `VinylTable.from_file`:

<Card>

In [22]:
VinylTable.from_file("data/barley.parquet")

DatabaseTable: barley
  yield   float64
  variety string
  year    int64
  site    string

</Card>

## Exploring relations

In the properties section action, you've already seen some of the most common ways to visualize relations:

- `print()` to view the AST
- `.schema()` to view the schema
- `.visualize()` to view the query plan
- `.execute()` to run the query and view it as text, pyarrow, or pandas objects
- `.to_sql()` to view the SQL-equivalent

In addition to these, there are a few other key methods to know:
- `.eda()` provides a quick exploratory data analysis of the relation, excluding its most common values
- `.chart()` provides a grammar of graphics interface to quickly visualize the relation

Let's view examples of each of these:

<Card>

In [23]:
weather.eda(topk=2)

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ name          ┃ type         ┃ nullable ┃ nulls ┃ non_nulls ┃ null_frac ┃ values                                         ┃ counts       ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string        │ string       │ boolean  │ int64 │ int64     │ float64   │ array<string>                                  │ array<int64> │
├───────────────┼──────────────┼──────────┼───────┼───────────┼───────────┼────────────────────────────────────────────────┼──────────────┤
│ date          │ timestamp(6) │ True     │     0 │      1461 │       0.0 │ ['2012-01-08 00:00:00', '2012-01-10 00:00:00'] │ [1, 1]       │
│ precipitation │ float64      │ True     │     0 │      1461 │       0.0 │ ['0.0', '0.3']                                 │ [838, 54]    │
│ temp_max      │ float64      │ True     │     0 │      1461 │       0.0 │ ['11.1', '14.4']                               │ [58, 49]     │
│ temp_min      │ float64      │ True     │     0 │      1461 │       0.0 │ ['6.1', '10.0']                                │ [66, 64]     │
│ wind          │ float64      │ True     │     0 │      1461 │       0.0 │ ['2.6', '3.0']                                 │ [76, 65]     │
│ weather       │ string       │ True     │     0 │      1461 │       0.0 │ ['sun', 'fog']                                 │ [714, 411]   │
└───────────────┴──────────────┴──────────┴───────┴───────────┴───────────┴────────────────────────────────────────────────┴──────────────┘

r0 := DatabaseTable: seattle_weather
  date          timestamp(6)
  precipitation float64
  temp_max      float64
  temp_min      float64
  wind          float64
  weather       string

r1 := Aggregate[r0]
  groups:
    weather: r0.weather
  metrics:
    Count(weather): Count(r0.weather)

r2 := Aggregate[r0]
  groups:
    wind: r0.wind
  metrics:
    Count(wind): Count(r0.wind)

r3 := Aggregate[r0]
  groups:
    temp_min: r0.temp_min
  metrics:
    Count(temp_min): Count(r0.temp_min)

r4 := Aggregate[r0]
  groups:
    temp_max: r0.temp_max
  metrics:
    Count(temp_max): Count(r0.temp_max)

r5 := Aggregate[r0]
  groups:
    date: r0.date
  metrics:
    Count(date): Count(r0.date)

r6 := Aggregate[r0]
  groups:
    precipitation: r0.precipitation
  metrics:
    Count(precipitation): Count(r0.precipitation)

r7 := Project[r0]
  isna: SearchedCase(cases=[IsNull(r0.weather)], results=[1], default=0)

r8 := Project[r0]
  isna: SearchedCase(cases=[IsNull(r0.wind)], results=[1], default=0)

r

</Card>

Charting relies on the awesome [lets-plot library](https://lets-plot.org/) from JetBrains

<Card>

In [24]:
from vinyl.chart import geom

weather.chart(geom.scatter, x=weather.date, y=weather.temp_max, interactive = False)

</Card>

::: {.callout-tip}

Charts made with Vinyl are interactive by default, but we turn that feature off so our docs site can render them properly.

:::